In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K
import tensorflow as tf

In [4]:
X_train = np.random.randint(3, high=32, size=(2000,16))
X_target = np.sort(X_train)
Y_train = np.copy(X_target)
Y_train[:,1:] = Y_train[:,:-1]
Y_train[:,0] = 1
X_train = np.concatenate([X_train, X_target], axis=-1)
X_train,X_test = X_train[1000:],X_train[:1000]
Y_train,Y_test = Y_train[1000:],Y_train[:1000]
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((1000, 32), (1000, 16), (1000, 32), (1000, 16))

In [5]:
data_input_dim = max(np.max(X_train), np.max(X_test)) + 1
data_input_dim

32

In [6]:
data_output_dim = max(np.max(Y_train), np.max(Y_test)) + 1
data_output_dim

32

In [7]:
class DNCCell(keras.layers.Layer):
    
    def __init__(self,
                 output_size,
                 word_size,
                 memory_size,
                 register_size=2,
                 write_head_count=2,
                 read_head_count=2,
                 enable_temporal=True,
                 enable_final_bias=True,
                 bypass_dropout_factor=None,
                 **kwargs):
        self.output_size = output_size
        self.word_size = word_size
        self.memory_size = memory_size
        self.register_size = register_size
        self.write_head_count = write_head_count
        self.read_head_count = read_head_count
        self.enable_temporal = enable_temporal
        self.enable_final_bias = enable_final_bias
        self.bypass_dropout_factor = bypass_dropout_factor
        self.state_size = (output_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (register_size * word_size,)
        self.state_size += (memory_size * word_size,)
        self.state_size += (read_head_count * memory_size,)
        self.state_size += (read_head_count * word_size,)
        self.state_size += (write_head_count * memory_size,)
        self.state_size += (memory_size,)
        if enable_temporal:
            self.state_size += (write_head_count * memory_size,)
            self.state_size += (write_head_count * memory_size**2,)
        super().__init__(**kwargs)
    
    def build(self, input_shape):
        read_vec_size = self.word_size * self.read_head_count
        controller_input_size = input_shape[-1]
        controller_input_size += read_vec_size
        controller_input_size += self.register_size*self.word_size
        controller_kernel_size = self.register_size*self.word_size*4
        controller_hidden_size = self.register_size*self.word_size
        interface_vec_size = read_vec_size
        interface_vec_size += self.read_head_count*2
        if self.enable_temporal:
            interface_vec_size += self.read_head_count*3
        interface_vec_size += self.write_head_count*self.word_size*3
        interface_vec_size += self.write_head_count*3
        self.kernel_controller_hidden = self.add_weight(
            shape=(controller_input_size, controller_kernel_size),
            initializer='glorot_normal', name='kernel_controller_hidden')
        self.bias_controller_hidden = self.add_weight(
            shape=(controller_kernel_size,),
            initializer='zeros', name='bias_controller_hidden')
        self.kernel_controller_output = self.add_weight(
            shape=(controller_hidden_size, self.output_size + interface_vec_size),
            initializer='glorot_normal', name='kernel_controller_output')
        self.kernel_read_vec_to_output = self.add_weight(
            shape=(read_vec_size, self.output_size),
            initializer='glorot_normal', name='kernel_read_vec_to_output')
        if self.enable_final_bias:
            self.bias_final_output = self.add_weight(
                shape=(self.output_size,),
                initializer='zeros', name='bias_final_output')
        super().build(input_shape)
    
    def call(self, inputs, states, training=None):
        
        def oneplus(x):
            return K.softplus(x) + 1.
        
        def similarity(m, k, b):
            dot = K.batch_dot(k, m, axes=2)
            m_len = K.sqrt(K.sum(K.square(m), axis=-1))
            k_len = K.sqrt(K.sum(K.square(k), axis=-1))
            mk_len = K.expand_dims(k_len, axis=-1) @ K.expand_dims(m_len, axis=-2)
            mk_len = K.switch(K.not_equal(mk_len, 0.), mk_len, K.ones_like(mk_len))
            cos_sim = dot / mk_len
            return K.softmax(cos_sim * K.expand_dims(b))
        
        def batch_invert_permutation(permutations):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            perm = tf.cast(permutations, tf.float32)
            dim = int(perm.get_shape()[-1])
            size = tf.cast(tf.shape(perm)[0], tf.float32)
            delta = tf.cast(tf.shape(perm)[-1], tf.float32)
            rg = tf.range(0, size * delta, delta, dtype=tf.float32)
            rg = tf.expand_dims(rg, 1)
            rg = tf.tile(rg, [1, dim])
            perm = tf.add(perm, rg)
            flat = tf.reshape(perm, [-1])
            perm = tf.invert_permutation(tf.cast(flat, tf.int32))
            perm = tf.reshape(perm, [-1, dim])
            return tf.subtract(perm, tf.cast(rg, tf.int32))

        def batch_gather(values, indices):
            # from https://github.com/deepmind/dnc/blob/master/dnc/util.py
            idx = tf.expand_dims(indices, -1)
            size = tf.shape(indices)[0]
            rg = tf.range(size, dtype=tf.int32)
            rg = tf.expand_dims(rg, -1)
            rg = tf.tile(rg, [1, int(indices.get_shape()[-1])])
            rg = tf.expand_dims(rg, -1)
            gidx = tf.concat([rg, idx], -1)
            return tf.gather_nd(values, gidx)
        
        _, register_s_last, register_h_last, memory_last, \
            read_weights_last, read_vec_last, write_weights_last, \
            usage_last, *temporal_states = states
        memory_last = K.reshape(memory_last,
            (-1,self.memory_size,self.word_size))
        read_weights_last = K.reshape(read_weights_last,
            (-1,self.read_head_count,self.memory_size))
        write_weights_last = K.reshape(write_weights_last,
            (-1,self.write_head_count,self.memory_size))
        if not self.enable_temporal:
            assert not temporal_states
        else:
            preced_last, link_last = temporal_states
            preced_last = K.reshape(preced_last,
                (-1,self.write_head_count,self.memory_size))
            link_last = K.reshape(link_last,
                (-1,self.write_head_count,self.memory_size,self.memory_size))
        
        # feeding controller with current input and last read vecs
        output = K.concatenate([inputs, read_vec_last, register_h_last])
        output = output @ self.kernel_controller_hidden
        output = output + self.bias_controller_hidden
        ctr_input_gate = output[:,:self.register_size*self.word_size]
        ctr_input_gate = K.sigmoid(ctr_input_gate)
        ctr_forget_gate = output[:,
            self.register_size*self.word_size:
            self.register_size*self.word_size*2]
        ctr_forget_gate = K.sigmoid(ctr_forget_gate)
        register_s = output[:,
            self.register_size*self.word_size*2:
            self.register_size*self.word_size*3]
        register_s = ctr_input_gate * K.tanh(register_s)
        register_s = register_s + ctr_forget_gate*register_s_last
        ctr_output_gate = output[:,
            self.register_size*self.word_size*3:
            self.register_size*self.word_size*4]
        ctr_output_gate = K.sigmoid(ctr_output_gate)
        register_h = ctr_output_gate * K.tanh(register_s)
        output = register_h @ self.kernel_controller_output
        
        # break down controller output into semi final output and interface vec
        interface_vec = output[:,self.output_size:]
        output = output[:,:self.output_size]
        if self.bypass_dropout_factor is not None:
            output = K.in_train_phase(K.dropout(
                output,self.bypass_dropout_factor),output,training=training)
        
        # break down interface vec
        interface_pos_last = 0
        interface_partition = []
        for interface_part_len in [
            self.read_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count,
            self.write_head_count * self.word_size,
            self.write_head_count * self.word_size,
            self.read_head_count,
            self.write_head_count,
            self.write_head_count,
            *([self.read_head_count * 3] if self.enable_temporal else [])]:
            interface_pos = interface_pos_last + interface_part_len
            interface_partition.append(
                interface_vec[:,interface_pos_last:interface_pos])
            interface_pos_last = interface_pos
        read_keys, read_stre, write_keys, write_stre, erase_vecs, \
            write_vecs, free_gates, alloc_gates, write_gates, \
            *temporal_interface_partition = interface_partition
        read_keys = K.reshape(read_keys,(-1,self.read_head_count,self.word_size))
        read_stre = oneplus(read_stre)
        write_keys = K.reshape(write_keys,(-1,self.write_head_count,self.word_size))
        write_stre = oneplus(write_stre)
        erase_vecs = K.reshape(erase_vecs,(-1,self.write_head_count,self.word_size))
        erase_vecs = K.sigmoid(erase_vecs)
        write_vecs = K.reshape(write_vecs,(-1,self.write_head_count,self.word_size))
        free_gates = K.expand_dims(free_gates)
        free_gates = K.sigmoid(free_gates)
        alloc_gates = K.expand_dims(alloc_gates)
        alloc_gates = K.sigmoid(alloc_gates)
        write_gates = K.expand_dims(write_gates)
        write_gates = K.sigmoid(write_gates)
        if not self.enable_temporal:
            assert not temporal_interface_partition
        else:
            read_modes, = temporal_interface_partition
            read_modes = K.reshape(read_modes,(-1,self.read_head_count,3))
            read_modes = K.softmax(read_modes,axis=-1)
        
        # compute allocation vector
        retention = K.prod(1.-(free_gates*read_weights_last), axis=-2)
        # https://github.com/deepmind/dnc/blob/master/dnc/addressing.py
        # according to the deepmind implementation,
        # only write weight is not differentiable
        write_weights_last_nograd = K.stop_gradient(write_weights_last)
        # reduce for multi-write-head, not presented in original papaer
        write_weights_last_nograd = 1.-K.prod(1.-write_weights_last_nograd, axis=-2)
        usage = ((usage_last+write_weights_last_nograd) - \
                usage_last*write_weights_last_nograd) * retention
        # loop for multi-write-head support
        mwh_write_gates = write_gates * alloc_gates
        mwh_usage = usage
        mwh_alloc = []
        for i in range(self.write_head_count):
            # quickfix for tf.cumprod grad bug
            # https://github.com/tensorflow/tensorflow/issues/3862
            usage_qfixed = 1e-6 + (1-1e-6) * mwh_usage
            usage_asc,usage_perm = tf.nn.top_k(1.-usage_qfixed,k=self.memory_size)
            usage_asc = 1.-usage_asc
            alloc_asc = (1.-usage_asc) * tf.cumprod(usage_asc,axis=-1,exclusive=True)
            alloc_perm = batch_invert_permutation(usage_perm)
            alloc = batch_gather(alloc_asc, alloc_perm)
            mwh_alloc.append(alloc)
            mwh_usage += (1.-mwh_usage) * mwh_write_gates[:,i,:] * alloc
        alloc = K.stack(mwh_alloc, axis=-2)
        
        # compute write weight
        write_sims = similarity(memory_last,write_keys,write_stre)
        write_weights = write_gates*(alloc_gates*alloc + (1.-alloc_gates)*write_sims)
        
        # compute precedence and temporal links
        if self.enable_temporal:
            write_weights_rep_h =  K.expand_dims(write_weights,axis=-1)
            write_weights_rep_v =  K.expand_dims(write_weights,axis=-2)
            link = (1.-(write_weights_rep_h+write_weights_rep_v))*link_last + \
                write_weights_rep_h * K.expand_dims(preced_last,axis=-2)
            link = link * (1.-tf.eye(self.memory_size))
            read_weights_last_rep = K.repeat_elements(
                K.expand_dims(read_weights_last,axis=-3),self.write_head_count,-3)
            # reduce sum for multi-write-head
            link_forward = K.sum(read_weights_last_rep @ link,axis=-3)
            link_backward = K.sum(K.permute_dimensions(link @ K.permute_dimensions(
                read_weights_last_rep,(0,1,3,2)),(0,1,3,2)),axis=-3)
            preced = (1.-K.sum(
                write_weights,axis=-1,keepdims=True))*preced_last + write_weights
        
        # update memory
        m_reset = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(erase_vecs,axis=-2)
        # reduce prod for multi-write-head
        m_keep = K.prod(1.-m_reset,axis=-3)
        m_new = K.expand_dims(write_weights,axis=-1) @ \
            K.expand_dims(write_vecs,axis=-2)
        # reduce sum for multi-write-head
        m_new = K.sum(m_new, axis=-3)
        memory = memory_last*m_keep + m_new
        
        # compute read weights and read vectors
        read_sims = similarity(memory,read_keys,read_stre)
        if not self.enable_temporal:
            read_weights = read_sims
        else:
            read_weights = read_modes[:,:,0:1] * read_sims + \
                            read_modes[:,:,1:2] * link_forward + \
                            read_modes[:,:,2:3] * link_backward
        read_vecs = read_weights @ memory
        read_vec = K.reshape(read_vecs, (-1,self.read_head_count*self.word_size))
        
        # compute final output from controller output and current read vecs
        output = output + read_vec @ self.kernel_read_vec_to_output
        if self.enable_final_bias:
            output = output + self.bias_final_output
        
        return output, [
            output,
            register_s,
            register_h,
            K.reshape(memory, (-1,self.memory_size*self.word_size)),
            K.reshape(read_weights, (-1,self.read_head_count*self.memory_size)),
            read_vec,
            K.reshape(write_weights, (-1,self.write_head_count*self.memory_size)),
            usage,
            *([
                K.reshape(preced, (-1,self.write_head_count*self.memory_size)),
                K.reshape(link, (-1,self.write_head_count*self.memory_size**2))
            ] if self.enable_temporal else [])]
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [8]:
X = X_inputs = keras.layers.Input((None,), dtype='int32')
X = keras.layers.Embedding(data_input_dim, 16, mask_zero=True)(X)
X = keras.layers.RNN(DNCCell(data_output_dim, 8, 16), return_sequences=True)(X)
X = keras.layers.Lambda(lambda x,s: x[:,s:,:],
    arguments={'s':X_train.shape[1]//2})(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.TimeDistributed(keras.layers.Dense(data_output_dim))(X)
X = keras.layers.Softmax()(X)
X = [keras.layers.Lambda(lambda x,s: x[:,s,:], name=f'o{i+1}',
    arguments={'s':i}, output_shape=(data_output_dim,))(X)
    for i in range(X_train.shape[1]//2)]
M = keras.Model(X_inputs, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 16)     512         input_1[0][0]                    
__________________________________________________________________________________________________
rnn_1 (RNN)                     (None, None, 32)     5472        embedding_1[0][0]                
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 32)     0           rnn_1[0][0]                      
__________________________________________________________________________________________________
activation

In [9]:
M.fit(X_train, np.split(Y_train,Y_train.shape[-1],axis=-1),
    validation_data=(X_test,np.split(Y_test,Y_test.shape[-1],axis=-1)),
    batch_size=8, epochs=50, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1)
])

Train on 1000 samples, validate on 1000 samples
Epoch 1/50
1000/1000 [==============================] - 57s 57ms/step - loss: 41.8907 - o1_loss: 2.4878 - o2_loss: 2.8150 - o3_loss: 2.8105 - o4_loss: 2.7679 - o5_loss: 2.6973 - o6_loss: 2.6396 - o7_loss: 2.6102 - o8_loss: 2.5698 - o9_loss: 2.5342 - o10_loss: 2.4889 - o11_loss: 2.4623 - o12_loss: 2.4957 - o13_loss: 2.5153 - o14_loss: 2.5670 - o15_loss: 2.6328 - o16_loss: 2.7963 - o1_acc: 0.6420 - o2_acc: 0.0380 - o3_acc: 0.0930 - o4_acc: 0.1580 - o5_acc: 0.2130 - o6_acc: 0.2390 - o7_acc: 0.2130 - o8_acc: 0.2140 - o9_acc: 0.2060 - o10_acc: 0.2240 - o11_acc: 0.2300 - o12_acc: 0.2520 - o13_acc: 0.2370 - o14_acc: 0.1980 - o15_acc: 0.1840 - o16_acc: 0.1440 - val_loss: 26.1775 - val_o1_loss: 1.0868 - val_o2_loss: 1.8479 - val_o3_loss: 1.8718 - val_o4_loss: 1.7814 - val_o5_loss: 1.7014 - val_o6_loss: 1.6123 - val_o7_loss: 1.5712 - val_o8_loss: 1.5564 - val_o9_loss: 1.5766 - val_o10_loss: 1.5686 - val_o11_loss: 1.5682 - val_o12_loss: 1.5582 - val

Epoch 7/50
1000/1000 [==============================] - 54s 54ms/step - loss: 2.0144 - o1_loss: 0.2262 - o2_loss: 0.2201 - o3_loss: 0.1528 - o4_loss: 0.1146 - o5_loss: 0.1281 - o6_loss: 0.1275 - o7_loss: 0.1067 - o8_loss: 0.1154 - o9_loss: 0.0923 - o10_loss: 0.1023 - o11_loss: 0.0863 - o12_loss: 0.0832 - o13_loss: 0.0950 - o14_loss: 0.0915 - o15_loss: 0.1083 - o16_loss: 0.1643 - o1_acc: 0.9170 - o2_acc: 0.9500 - o3_acc: 0.9750 - o4_acc: 0.9870 - o5_acc: 0.9780 - o6_acc: 0.9850 - o7_acc: 0.9870 - o8_acc: 0.9810 - o9_acc: 0.9870 - o10_acc: 0.9840 - o11_acc: 0.9840 - o12_acc: 0.9870 - o13_acc: 0.9850 - o14_acc: 0.9840 - o15_acc: 0.9840 - o16_acc: 0.9820 - val_loss: 1.5917 - val_o1_loss: 0.1923 - val_o2_loss: 0.1715 - val_o3_loss: 0.1409 - val_o4_loss: 0.1040 - val_o5_loss: 0.0919 - val_o6_loss: 0.0811 - val_o7_loss: 0.0914 - val_o8_loss: 0.0854 - val_o9_loss: 0.0763 - val_o10_loss: 0.0785 - val_o11_loss: 0.0518 - val_o12_loss: 0.0635 - val_o13_loss: 0.0693 - val_o14_loss: 0.0818 - val_o15

Epoch 13/50
1000/1000 [==============================] - 54s 54ms/step - loss: 0.5222 - o1_loss: 0.1225 - o2_loss: 0.0995 - o3_loss: 0.0332 - o4_loss: 0.0176 - o5_loss: 0.0236 - o6_loss: 0.0219 - o7_loss: 0.0199 - o8_loss: 0.0290 - o9_loss: 0.0194 - o10_loss: 0.0196 - o11_loss: 0.0182 - o12_loss: 0.0161 - o13_loss: 0.0199 - o14_loss: 0.0164 - o15_loss: 0.0211 - o16_loss: 0.0244 - o1_acc: 0.9580 - o2_acc: 0.9780 - o3_acc: 0.9960 - o4_acc: 1.0000 - o5_acc: 0.9970 - o6_acc: 0.9990 - o7_acc: 1.0000 - o8_acc: 0.9960 - o9_acc: 1.0000 - o10_acc: 0.9990 - o11_acc: 0.9990 - o12_acc: 0.9990 - o13_acc: 0.9970 - o14_acc: 0.9980 - o15_acc: 0.9980 - o16_acc: 1.0000 - val_loss: 0.4541 - val_o1_loss: 0.1253 - val_o2_loss: 0.0754 - val_o3_loss: 0.0347 - val_o4_loss: 0.0219 - val_o5_loss: 0.0165 - val_o6_loss: 0.0140 - val_o7_loss: 0.0235 - val_o8_loss: 0.0191 - val_o9_loss: 0.0175 - val_o10_loss: 0.0178 - val_o11_loss: 0.0096 - val_o12_loss: 0.0132 - val_o13_loss: 0.0132 - val_o14_loss: 0.0179 - val_o1

Epoch 19/50
1000/1000 [==============================] - 52s 52ms/step - loss: 0.8957 - o1_loss: 0.1054 - o2_loss: 0.0724 - o3_loss: 0.0282 - o4_loss: 0.0320 - o5_loss: 0.0235 - o6_loss: 0.0387 - o7_loss: 0.0444 - o8_loss: 0.0561 - o9_loss: 0.0869 - o10_loss: 0.0806 - o11_loss: 0.0956 - o12_loss: 0.0716 - o13_loss: 0.0549 - o14_loss: 0.0325 - o15_loss: 0.0445 - o16_loss: 0.0284 - o1_acc: 0.9740 - o2_acc: 0.9860 - o3_acc: 0.9920 - o4_acc: 0.9870 - o5_acc: 0.9940 - o6_acc: 0.9940 - o7_acc: 0.9870 - o8_acc: 0.9880 - o9_acc: 0.9820 - o10_acc: 0.9840 - o11_acc: 0.9780 - o12_acc: 0.9870 - o13_acc: 0.9910 - o14_acc: 0.9920 - o15_acc: 0.9890 - o16_acc: 0.9930 - val_loss: 3.0208 - val_o1_loss: 0.8803 - val_o2_loss: 0.1981 - val_o3_loss: 0.1423 - val_o4_loss: 0.1173 - val_o5_loss: 0.0880 - val_o6_loss: 0.1012 - val_o7_loss: 0.1459 - val_o8_loss: 0.1576 - val_o9_loss: 0.2004 - val_o10_loss: 0.1977 - val_o11_loss: 0.1573 - val_o12_loss: 0.1467 - val_o13_loss: 0.1107 - val_o14_loss: 0.1082 - val_o1

Epoch 25/50
1000/1000 [==============================] - 51s 51ms/step - loss: 0.3458 - o1_loss: 0.1206 - o2_loss: 0.0738 - o3_loss: 0.0272 - o4_loss: 0.0089 - o5_loss: 0.0106 - o6_loss: 0.0084 - o7_loss: 0.0090 - o8_loss: 0.0124 - o9_loss: 0.0106 - o10_loss: 0.0092 - o11_loss: 0.0081 - o12_loss: 0.0085 - o13_loss: 0.0097 - o14_loss: 0.0078 - o15_loss: 0.0098 - o16_loss: 0.0112 - o1_acc: 0.9400 - o2_acc: 0.9920 - o3_acc: 0.9980 - o4_acc: 1.0000 - o5_acc: 1.0000 - o6_acc: 1.0000 - o7_acc: 1.0000 - o8_acc: 0.9990 - o9_acc: 0.9990 - o10_acc: 1.0000 - o11_acc: 1.0000 - o12_acc: 1.0000 - o13_acc: 0.9990 - o14_acc: 1.0000 - o15_acc: 1.0000 - o16_acc: 1.0000 - val_loss: 0.4421 - val_o1_loss: 0.1557 - val_o2_loss: 0.0757 - val_o3_loss: 0.0506 - val_o4_loss: 0.0175 - val_o5_loss: 0.0150 - val_o6_loss: 0.0103 - val_o7_loss: 0.0126 - val_o8_loss: 0.0117 - val_o9_loss: 0.0118 - val_o10_loss: 0.0116 - val_o11_loss: 0.0095 - val_o12_loss: 0.0127 - val_o13_loss: 0.0116 - val_o14_loss: 0.0101 - val_o1

Epoch 31/50
1000/1000 [==============================] - 51s 51ms/step - loss: 0.3398 - o1_loss: 0.1174 - o2_loss: 0.0738 - o3_loss: 0.0271 - o4_loss: 0.0087 - o5_loss: 0.0104 - o6_loss: 0.0081 - o7_loss: 0.0088 - o8_loss: 0.0121 - o9_loss: 0.0102 - o10_loss: 0.0090 - o11_loss: 0.0080 - o12_loss: 0.0086 - o13_loss: 0.0095 - o14_loss: 0.0078 - o15_loss: 0.0095 - o16_loss: 0.0109 - o1_acc: 0.9410 - o2_acc: 0.9910 - o3_acc: 0.9980 - o4_acc: 1.0000 - o5_acc: 1.0000 - o6_acc: 1.0000 - o7_acc: 1.0000 - o8_acc: 0.9980 - o9_acc: 0.9990 - o10_acc: 1.0000 - o11_acc: 1.0000 - o12_acc: 1.0000 - o13_acc: 1.0000 - o14_acc: 1.0000 - o15_acc: 1.0000 - o16_acc: 1.0000 - val_loss: 0.4372 - val_o1_loss: 0.1525 - val_o2_loss: 0.0759 - val_o3_loss: 0.0498 - val_o4_loss: 0.0174 - val_o5_loss: 0.0149 - val_o6_loss: 0.0102 - val_o7_loss: 0.0123 - val_o8_loss: 0.0117 - val_o9_loss: 0.0115 - val_o10_loss: 0.0116 - val_o11_loss: 0.0094 - val_o12_loss: 0.0132 - val_o13_loss: 0.0114 - val_o14_loss: 0.0096 - val_o1

Epoch 37/50
1000/1000 [==============================] - 51s 51ms/step - loss: 0.3399 - o1_loss: 0.1174 - o2_loss: 0.0739 - o3_loss: 0.0270 - o4_loss: 0.0087 - o5_loss: 0.0104 - o6_loss: 0.0081 - o7_loss: 0.0088 - o8_loss: 0.0121 - o9_loss: 0.0102 - o10_loss: 0.0090 - o11_loss: 0.0079 - o12_loss: 0.0086 - o13_loss: 0.0095 - o14_loss: 0.0078 - o15_loss: 0.0096 - o16_loss: 0.0109 - o1_acc: 0.9410 - o2_acc: 0.9910 - o3_acc: 0.9980 - o4_acc: 1.0000 - o5_acc: 1.0000 - o6_acc: 1.0000 - o7_acc: 1.0000 - o8_acc: 0.9980 - o9_acc: 0.9990 - o10_acc: 1.0000 - o11_acc: 1.0000 - o12_acc: 1.0000 - o13_acc: 1.0000 - o14_acc: 1.0000 - o15_acc: 1.0000 - o16_acc: 1.0000 - val_loss: 0.4370 - val_o1_loss: 0.1525 - val_o2_loss: 0.0760 - val_o3_loss: 0.0498 - val_o4_loss: 0.0174 - val_o5_loss: 0.0149 - val_o6_loss: 0.0102 - val_o7_loss: 0.0123 - val_o8_loss: 0.0117 - val_o9_loss: 0.0115 - val_o10_loss: 0.0116 - val_o11_loss: 0.0094 - val_o12_loss: 0.0130 - val_o13_loss: 0.0114 - val_o14_loss: 0.0096 - val_o1

Epoch 43/50
1000/1000 [==============================] - 51s 51ms/step - loss: 0.3399 - o1_loss: 0.1174 - o2_loss: 0.0739 - o3_loss: 0.0270 - o4_loss: 0.0087 - o5_loss: 0.0104 - o6_loss: 0.0081 - o7_loss: 0.0088 - o8_loss: 0.0121 - o9_loss: 0.0102 - o10_loss: 0.0090 - o11_loss: 0.0079 - o12_loss: 0.0086 - o13_loss: 0.0095 - o14_loss: 0.0078 - o15_loss: 0.0096 - o16_loss: 0.0109 - o1_acc: 0.9410 - o2_acc: 0.9910 - o3_acc: 0.9980 - o4_acc: 1.0000 - o5_acc: 1.0000 - o6_acc: 1.0000 - o7_acc: 1.0000 - o8_acc: 0.9980 - o9_acc: 0.9990 - o10_acc: 1.0000 - o11_acc: 1.0000 - o12_acc: 1.0000 - o13_acc: 1.0000 - o14_acc: 1.0000 - o15_acc: 1.0000 - o16_acc: 1.0000 - val_loss: 0.4370 - val_o1_loss: 0.1525 - val_o2_loss: 0.0760 - val_o3_loss: 0.0498 - val_o4_loss: 0.0174 - val_o5_loss: 0.0149 - val_o6_loss: 0.0102 - val_o7_loss: 0.0123 - val_o8_loss: 0.0117 - val_o9_loss: 0.0115 - val_o10_loss: 0.0116 - val_o11_loss: 0.0094 - val_o12_loss: 0.0130 - val_o13_loss: 0.0114 - val_o14_loss: 0.0096 - val_o1

Epoch 49/50
1000/1000 [==============================] - 51s 51ms/step - loss: 0.3399 - o1_loss: 0.1174 - o2_loss: 0.0739 - o3_loss: 0.0270 - o4_loss: 0.0087 - o5_loss: 0.0104 - o6_loss: 0.0081 - o7_loss: 0.0088 - o8_loss: 0.0121 - o9_loss: 0.0102 - o10_loss: 0.0090 - o11_loss: 0.0079 - o12_loss: 0.0086 - o13_loss: 0.0095 - o14_loss: 0.0078 - o15_loss: 0.0096 - o16_loss: 0.0109 - o1_acc: 0.9410 - o2_acc: 0.9910 - o3_acc: 0.9980 - o4_acc: 1.0000 - o5_acc: 1.0000 - o6_acc: 1.0000 - o7_acc: 1.0000 - o8_acc: 0.9980 - o9_acc: 0.9990 - o10_acc: 1.0000 - o11_acc: 1.0000 - o12_acc: 1.0000 - o13_acc: 1.0000 - o14_acc: 1.0000 - o15_acc: 1.0000 - o16_acc: 1.0000 - val_loss: 0.4370 - val_o1_loss: 0.1525 - val_o2_loss: 0.0760 - val_o3_loss: 0.0498 - val_o4_loss: 0.0174 - val_o5_loss: 0.0149 - val_o6_loss: 0.0102 - val_o7_loss: 0.0123 - val_o8_loss: 0.0117 - val_o9_loss: 0.0115 - val_o10_loss: 0.0116 - val_o11_loss: 0.0094 - val_o12_loss: 0.0130 - val_o13_loss: 0.0114 - val_o14_loss: 0.0096 - val_o1